# Selenium
**Selenium** es una _Web Browser Automation Tool_, es decir, permite abrir un buscador y realizar tareas que "imiten" el comportamiento humano (clicks, scrolling, *búsquedas*, etc).  
Link a la documentación de Selinium: [https://selenium-python.readthedocs.io/](https://selenium-python.readthedocs.io/)  
Link a la dicumentación y descarga de chromedriver: [https://chromedriver.chromium.org/home](https://chromedriver.chromium.org/home)

In [1]:
# !pip install selenium

In [2]:
# !apt-get install -y chromium-browser

In [3]:
""" !apt-get update
# !apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') """

" !apt-get update\n# !apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\nimport sys\nsys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') "

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import pandas as pd
import time

In [5]:
from selenium import webdriver

def configure_driver():
    # Add additional Options to the webdriver
    options = webdriver.ChromeOptions()
    # add the argument and make the browser Headless.
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox') # only colab
    # options.add_argument('--disable-dev-shm-usage') # only colab
    # Handle the service
    # service = Service(executable_path=r'/usr/bin/chromedriver')
    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    driver = webdriver.Chrome(options=options)

    #driver = webdriver.Firefox()
    return driver

In [6]:
def getCourses(driver, total_clicks):
    # https://www.pluralsight.com/search?q=web%20scraping&categories=course
    # Step 1: Go to pluralsight.com, category section with selected search keyword
    # driver.get("https://www.pluralsight.com/search?q={search_keyword}&categories=course")
    driver.get("https://www.pluralsight.com/search?categories=course&sort=displayDate")

    timeout = 15

    time.sleep(timeout)
    count = 0
    for k in tqdm(range(total_clicks)):
        try:
            element = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, "//a[@id='search-results-section-load-more']")))
            element.click()
            count = 0
        except:
            count += 1
            time.sleep(timeout)
            print("Error...")
            if count > 5:
                break

    data = []
    # wait for the element to load
    try:
        WebDriverWait(driver, timeout*10).until(lambda s: s.find_element(By.ID, "search-results-category-target").is_displayed())
    except TimeoutException:
        print("Timeout :(")
        return None
    
    # Step 2: Create a parse tree of page sources after searching
    soup = BeautifulSoup(driver.page_source, "lxml")

    # Step 3: Iterate over the search result and fetch the course
    print(len(soup.select("div.search-results-page")))
    for course_page in soup.select("div.search-results-page"):
        for course in course_page.select("div.search-result"):
            title_selector = "div.search-result__info div.search-result__title a"
            title_url_selector = "div.search-result__info div.search-result__title a"
            author_selector = "div.search-result__details div.search-result__author"
            level_selector = "div.search-result__details div.search-result__level"
            length_selector = "div.search-result__details div.search-result__length"
            date_selector = "div.search-result__details div.search-result__date"
            icon_selector = "div.search-result__icon img"

            data.append({
                "title" : course.select_one(title_selector).text,
                "title_url_selector" : course.select_one(title_url_selector).attrs["href"],
                "date": course.select_one(date_selector).text,
                "author" : course.select_one(author_selector).text,
                "level" : course.select_one(level_selector).text,
                "length" : course.select_one(length_selector).text,
                "icon": course.select_one(icon_selector).attrs["src"]
            })
    return data

total_clicks = 300
# create the driver object.
driver = configure_driver()
data = getCourses(driver, total_clicks)

# close the driver.
driver.close()

  1%|          | 2/300 [00:30<1:15:50, 15.27s/it]

Error...


  4%|▎         | 11/300 [01:29<57:08, 11.86s/it] 

Error...


  4%|▍         | 13/300 [01:59<1:04:16, 13.44s/it]

Error...


In [9]:
df_data = pd.DataFrame.from_dict(data)
df_data

,title,title_url_selector,date,author,level,length,icon
0,Introduction to Microsoft Copilot,https://www.pluralsight.com/courses/microsoft-...,Sep 1 2023,by Vlad Catrinescu,Beginner,1h 26m,https://pluralsight.imgix.net/course-images/mi...
1,Introduction to the Microsoft Collaboration Co...,https://www.pluralsight.com/courses/msft-colla...,Sep 1 2023,by Jeff Brown,Beginner,29m,https://pluralsight.imgix.net/course-images/ms...
2,Introduction to Emotional Intelligence,https://www.pluralsight.com/courses/introducti...,Sep 1 2023,by Jason Edleman,Beginner,1h 27m,https://pluralsight.imgix.net/course-images/in...
3,Functional Programming in Rust 2021,https://www.pluralsight.com/courses/rust-2021-...,Sep 1 2023,by Shaun Meyer,Intermediate,1h 14m,https://pluralsight.imgix.net/course-images/ru...
4,Key Concepts: Data Architecture Design,https://www.pluralsight.com/courses/data-archi...,Sep 1 2023,by Wilvie Anora,Beginner,1h 27m,https://pluralsight.imgix.net/course-images/da...
5,Microsoft Purview: Manage and Protect On-premi...,https://www.pluralsight.com/courses/microsoft-...,Aug 31 2023,by Liam Cleary,Intermediate,2h 45m,https://pluralsight.imgix.net/course-images/mi...
6,Apex Academy: SOQL and Working with Databases,https://www.pluralsight.com/courses/apex-acade...,Aug 31 2023,by Scott Lee,Beginner,1h 17m,https://pluralsight.imgix.net/course-images/ap...
7,REST Assured Deep Dive,https://www.pluralsight.com/courses/rest-assur...,Aug 31 2023,by Andrejs Doronins,Advanced,2h 11m,https://pluralsight.imgix.net/course-images/re...
8,Apex Academy: Asynchronous Apex and Going Beyo...,https://www.pluralsight.com/courses/apex-acade...,Aug 31 2023,by Scott Lee,Beginner,1h 31m,https://pluralsight.imgix.net/course-images/ap...
9,Using Generative AI to Troubleshoot Linux,https://www.pluralsight.com/courses/linux-usin...,Aug 31 2023,by Tim Warner,Intermediate,1h 16m,https://pluralsight.imgix.net/course-images/li...


In [12]:
df_data.to_csv("dataset_cursos_{}.csv".format(total_clicks))

# **Handling AJAX Loading and Infinite Loading**
Sometimes, fetching content from dynamic sites is actually straightforward, as they are highly dependent on API calls. In asynchronous loading, most of the time, data is loaded by making GET and POST requests; you can watch these API calls in the Network tab of Developer Tools.
https://www.premierleague.com/players


In [ ]:
import requests
import json

In [ ]:
# https://footballapi.pulselive.com/football/players?pageSize=30&compSeasons=489&altIds=true&page=0&type=player&id=-1&compSeasonId=489

In [ ]:
# API url
url = "https://footballapi.pulselive.com/football/players"
# Headers required for making a GET request
# It is a good practice to provide headers with each request.
headers = {
    "account": "premierleague",
    "content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "DNT": "1",
    "Origin": "https://www.premierleague.com",
    "Referer": "https://www.premierleague.com/players",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"
}

# Query parameters required to make get request
queryParams = {
    "pageSize": 100,
    "compSeasons": 578,
    "altIds": True,
    "page": 0,
    "type": "player",
    "compSeasonId": 578,
    "cl":2,
}

In [ ]:
# Sending the request with url, headers, and query params
response = requests.get(url=url, headers=headers, params=queryParams)

In [ ]:
response.status_code

200

In [ ]:
response.text

<bound method Response.json of <Response [200]>>

In [ ]:
# if response status code is 200 OK, then
if response.status_code == 200:
  data = json.loads(response.text)
  for player in data["content"]:
    print({
        "name": player["name"]["display"],
        "nationalTeam": player["nationalTeam"]["country"],
        "position": player["info"]["positionInfo"]
    })

{'name': 'Max Aarons', 'nationalTeam': 'England', 'position': 'Right Full Back'}
{'name': 'Zach Abbott', 'nationalTeam': 'England', 'position': 'Defender'}
{'name': 'Terry Ablade', 'nationalTeam': 'Finland', 'position': 'Centre Striker'}
{'name': 'Tyler Adams', 'nationalTeam': 'United States', 'position': 'Midfielder'}
{'name': 'Tosin Adarabioyo', 'nationalTeam': 'England', 'position': 'Centre Central Defender'}
{'name': 'Tayo Adaramola', 'nationalTeam': 'Ireland', 'position': 'Left Full Back'}
{'name': 'Elijah Adebayo', 'nationalTeam': 'England', 'position': 'Centre Striker'}
{'name': 'Valintino Adedokun', 'nationalTeam': 'Ireland', 'position': 'Defender'}
{'name': 'Simon Adingra', 'nationalTeam': 'Cote D’Ivoire', 'position': 'Left Winger'}
{'name': 'Adler Nascimento', 'nationalTeam': 'Portugal', 'position': 'Left Winger'}
{'name': 'Adrián', 'nationalTeam': 'Spain', 'position': 'Goalkeeper'}
{'name': 'Daniel Adu-Adjei', 'nationalTeam': 'England', 'position': 'Midfielder'}
{'name': 'Na